# Conveyor Model Optimizer

Customers always enter in the first slot of the server 

In [1]:
using JuMP
using HiGHS
using Juniper

include("utils.jl") # demand_generator_mat, printTable, plotData

plotData (generic function with 1 method)

In [20]:
# Parameters
horiz = 10        # total horizon

# bounds
YM = 10              # max buffer legnth before dropping calls
XM = 6               # max queue length 
phiM = 4             # max adimission to queue
serM = 3            # number of servers
tserM = 3            # max service time


# iniital conditions
X0 = 3
Y0 = 2
L0 = 0
Z0 = 0

createDemands = false
if createDemands
    d_mat = demand_generator_mat(1, 20, 5, "uniform", 1)
    a_mat = demand_generator_mat(1, 20, 1,"uniform",0.5)
else
    d_fn = "..//CC_simple//d_mat_Thu_22_May_2025_19_39_50.txt";
    a_fn = "..//CC_simple//a_mat_Thu_22_May_2025_19_39_50.txt";
    d_mat = DelimitedFiles.readdlm(d_fn);
    a_mat = DelimitedFiles.readdlm(a_fn);
    
end

d = d_mat[1:horiz, 1];  # demand for incoming calls
a = a_mat[1:horiz, 1];  # abandonment for calls

In [21]:
J = zeros(horiz)            # cost function
X = zeros(Int, horiz+1)     # current number of customers in queue x(k)
Y = zeros(Int, horiz+1)     # current number of customers in buffer y(k)  
Z = zeros(Int, horiz+1)     # custumers served  
L = zeros(Int, horiz+1)     # number of customers lost

n = zeros(Int, horiz+1)     # number of empty slots in the queue
Q = zeros(Int, horiz+1)     # custumers entering queue
dr = zeros(Int, horiz+1)    # dropped due to full buffer

phi = zeros(Int, horiz+1)     # number of customers admitted to queue
Cin = zeros(Int, horiz+1)     # number of customers entering server
Cout = zeros(Int, horiz+1)    # number of customers leaving server

S = zeros(Int, horiz+1)                 # number of active servers
Sl = zeros(Int, horiz+1)                # number of free servers
Sst = zeros(Bool, horiz+1, serM)        # server status (0 - free, 1 - busy)
Sc = zeros(Bool, horiz+1, serM, tserM)  # server conveyor
Sin = zeros(Bool, horiz+1, serM, tserM) # server input

Saux = zeros(Bool, horiz+1, serM)       # auxiliary server variable
b1_opt = zeros(horiz)
b2_opt = zeros(horiz)

default_input = zeros(Bool, serM, tserM)  # default input for servers
for i in 1:serM
    default_input[i, 1] = 1
end

transition_matrix = zeros(Bool, tserM, tserM)
for i in 1:tserM-1
    transition_matrix[i, i+1] = 1
end

In [22]:
# initial conditions
X[1] = X0
Y[1] = Y0
L[1] = L0
Z[1] = Z0

0

In [23]:
for t in 1:horiz
    cc_lin_conv = Model(HiGHS.Optimizer)
    set_silent(cc_lin_conv)
    
    # For the integer constraint on variable b
    M1 = max(d[t], YM)+10; # must be larger than d[t] and n[t] 
    @variable(cc_lin_conv, b1, Bin);

    M2 = max(X[t], serM)+10; # must be larger than x[k] and Sl[k]
    @variable(cc_lin_conv, b2, Bin);

    @variable(cc_lin_conv, 0 <= XL[1:2] <= XM, Int)    
    @variable(cc_lin_conv, 0 <= YL[1:2] <= YM, Int)     
    @variable(cc_lin_conv, 0 <= ZL[1:2], Int)  
    @variable(cc_lin_conv, 0 <= LL[1:2], Int)         
                           
    @variable(cc_lin_conv, 0 <= nL <= YM, Int) 
    @variable(cc_lin_conv, 0 <= QL, Int)      
    @variable(cc_lin_conv, 0 <= drL, Int)   
    
    @variable(cc_lin_conv, 0 <= phiL <= phiM, Int)   
    @variable(cc_lin_conv, 0 <= CinL <= serM, Int)   
    @variable(cc_lin_conv, 0 <= CoutL[1:2] <= serM, Int)    

    @variable(cc_lin_conv, 0 <= SL <= serM, Int)  
    @variable(cc_lin_conv, 0 <= SlL <= serM, Int)  
    @variable(cc_lin_conv, 0 <= SstL[1:serM], Bin)  
    @variable(cc_lin_conv, 0 <= ScL[1:2, 1:serM, 1:tserM], Bin) 
    @variable(cc_lin_conv, 0 <= SinL[1:serM, 1:tserM], Bin)
    @variable(cc_lin_conv, 0 <= SauxL[1:serM], Bin)

        
    # Initial conditions of buffer and queue for each optimization
    @constraint(cc_lin_conv, XL[1] == X[t])
    @constraint(cc_lin_conv, YL[1] == Y[t])
    @constraint(cc_lin_conv, ZL[1] == Z[t])
    @constraint(cc_lin_conv, LL[1] == L[t])
    @constraint(cc_lin_conv, CoutL[1] == Cout[t])
    @constraint(cc_lin_conv, ScL[1, :, :] == Sc[t, :, :])  # server conveyor status

    # Problem constraints  
    @constraint(cc_lin_conv, XL[2] == XL[1] + phiL - a[t] - CinL)
    @constraint(cc_lin_conv, YL[2] == YL[1] + QL - phiL)
    @constraint(cc_lin_conv, ZL[2] == ZL[1] + CoutL[1])
    @constraint(cc_lin_conv, LL[2] == LL[1] + drL + a[t])

    @constraint(cc_lin_conv, nL == YM - YL[1] + phiL)  # number of empty slots in the queue
    @constraint(cc_lin_conv, QL <= d[t])
    @constraint(cc_lin_conv, QL <= nL)
    @constraint(cc_lin_conv, QL >= d[t]-M1*b1)
    @constraint(cc_lin_conv, QL >= nL-(1-b1)*M1)  

    @constraint(cc_lin_conv, drL >= d[t]-nL)         
    @constraint(cc_lin_conv, drL <= d[t]-QL) 

    @constraint(cc_lin_conv, SL == serM)              
    @constraint(cc_lin_conv, SstL == sum(ScL[1,:,:], dims=2))  
    @constraint(cc_lin_conv, SlL == SL - sum(SstL))    
    @constraint(cc_lin_conv, [i=1:serM, j=1:tserM], SinL[i, j] == default_input[i, j] .* SauxL[i])
    @constraint(cc_lin_conv, ScL[2, :, :] == ScL[1, :, :]*transition_matrix + SinL)

    @constraint(cc_lin_conv, CinL <= XL[1]-a[t])
    @constraint(cc_lin_conv, CinL <= SlL)
    @constraint(cc_lin_conv, CinL >= XL[1]-a[t]-M2*b2 )
    @constraint(cc_lin_conv, CinL >= SlL-(1-b2)*M2)  
    @constraint(cc_lin_conv, CoutL[2] == sum(ScL[1,:,tserM]))  

    @constraint(cc_lin_conv, SauxL <= ones(Bool, serM) - SstL)  
    @constraint(cc_lin_conv, CinL == sum(SauxL))  

    # Objective function
    # @objective(cc_lin_conv, Max, CoutL[1] - LL[1] - phiL) 
    # @objective(cc_lin_conv, Max, Cin[1] - LL[1]) 
    @objective(cc_lin_conv, Max, phiL) # simplified version where phi will always be as big as possible

    JuMP.optimize!(cc_lin_conv)

    if t == 6 || t == 7
        println("constraint 1: ", JuMP.value(XL[1])-a[t])
        println("constraint 2: ", serM - sum(sum(JuMP.value.(Sc[t, :, :]), dims=2)))
        println("constraint 3: ", max(JuMP.value(X[1]), serM)+10)

        println("Cin: ", JuMP.value(CinL))
        println("Sl: ", JuMP.value(SlL))
        println("X: ", JuMP.value(XL[1]))

        println("b2: ", JuMP.value(b2))
        println("M2: ", JuMP.value(M2))
        
        print("Sc")
        display(JuMP.value.(ScL[1,:,:]))
        print("Sst")
        display(JuMP.value.(SstL))
        print("Saux")
        display(JuMP.value.(SauxL))
    end
    
    status = termination_status(cc_lin_conv)
    if (status == MOI.OPTIMAL || status == MOI.LOCALLY_SOLVED || status==MOI.ALMOST_LOCALLY_SOLVED) && has_values(cc_lin_conv)
        # Save computed values
        X[t+1] = JuMP.value(XL[2]);
        Y[t+1] = JuMP.value(YL[2]);
        Z[t+1] = JuMP.value(ZL[2]);
        L[t+1] = JuMP.value(LL[2]);

        n[t] = JuMP.value(nL);
        Q[t] = JuMP.value(QL);  
        dr[t] = JuMP.value(drL);
        
        phi[t] = JuMP.value(phiL);
        Cin[t] = JuMP.value(CinL);
        Cout[t+1] = JuMP.value(CoutL[2]);

        S[t] = JuMP.value(SL);
        Sl[t] = JuMP.value(SlL);
        Sst[t, :] = JuMP.value.(SstL);
        Sc[t+1, :, :] = JuMP.value.(ScL[2, :, :]);
        Sin[t, :, :] = JuMP.value.(SinL);
        Saux[t, :] = JuMP.value.(SauxL);

        b1_opt[t] = JuMP.value(b1);
        b2_opt[t] = JuMP.value(b2);

        J[t] = objective_value(cc_lin_conv) # Repeated line

        # Display results
        # println("Optimal solution:")
        println("Objective value = ", objective_value(cc_lin_conv))
    else
        println(t, " ", status)
    end
end

Objective value = 2.0
Objective value = 4.0
Objective value = 3.0
Objective value = 1.0
Objective value = 4.0
constraint 1: 4.0
constraint 2: 0
constraint 3: 13
Cin: 0.0
Sl: 0.0
X: 6.0
b2: 1.0
M2: 16
Sc

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 1.0  0.0  0.0
 1.0  0.0  0.0

Sst

3-element Vector{Float64}:
 1.0
 1.0
 1.0

Saux

...

Sst

3-element Vector{Float64}:
 1.0
 1.0
 1.0

Saux

3-element Vector{Float64}:
 0.0
 0.0
 0.0

Objective value = 1.0
Objective value = 1.0
Objective value = 4.0
Objective value = 1.0


In [24]:
data = hcat(1:horiz, d, dr[1:horiz,1], n[1:horiz,1], Q[1:horiz,1], Y[1:horiz,1], phi[1:horiz, 1], X[1:horiz,1], Cin[1:horiz,1], Cout[1:horiz,1], a)
header = (["t", "d", "dr", "n", "q", "Y", "phi", "X", "Cin", "Cout", "a"]);
printTable(data, header)

Results
┌────────┬────────┬───────┬────────┬───────┬────────┬───────┬───────┬───────┬───────┬───────┐
│      t │      d │    dr │      n │     q │      Y │   phi │     X │   Cin │  Cout │     a │
├────────┼────────┼───────┼────────┼───────┼────────┼───────┼───────┼───────┼───────┼───────┤
│  1.000 │  7.000 │ 0.000 │ 10.000 │ 7.000 │  2.000 │ 2.000 │ 3.000 │ 3.000 │ 0.000 │ 0.000 │
│  2.000 │  3.000 │ 0.000 │  7.000 │ 3.000 │  7.000 │ 4.000 │ 2.000 │ 0.000 │ 0.000 │ 1.000 │
│  3.000 │  3.000 │ 0.000 │  7.000 │ 3.000 │  6.000 │ 3.000 │ 5.000 │ 0.000 │ 0.000 │ 2.000 │
│  4.000 │  7.000 │ 2.000 │  5.000 │ 5.000 │  6.000 │ 1.000 │ 6.000 │ 0.000 │ 0.000 │ 1.000 │
│  5.000 │ 12.000 │ 8.000 │  4.000 │ 4.000 │ 10.000 │ 4.000 │ 6.000 │ 3.000 │ 3.000 │ 1.000 │
│  6.000 │  9.000 │ 7.000 │  2.000 │ 2.000 │ 10.000 │ 2.000 │ 6.000 │ 0.000 │ 0.000 │ 2.000 │
│  7.000 │  6.000 │ 5.000 │  1.000 │ 1.000 │ 10.000 │ 1.000 │ 6.000 │ 0.000 │ 0.000 │ 1.000 │
│  8.000 │  3.000 │ 2.000 │  1.000 │ 1.000 │ 10.000 

In [12]:
data = hcat(1:horiz, S[1:horiz,1], Sst[1:horiz,1], Sl[1:horiz,1], Saux[1:horiz,1])
header = (["t", "S", "Sst", "Sl", "Saux"]);
printTable(data, header)

Results
┌────────┬───────┬───────┬───────┬───────┐
│      t │     S │   Sst │    Sl │  Saux │
├────────┼───────┼───────┼───────┼───────┤
│  1.000 │ 2.000 │ 0.000 │ 2.000 │ 1.000 │
│  2.000 │ 2.000 │ 1.000 │ 0.000 │ 0.000 │
│  3.000 │ 2.000 │ 1.000 │ 0.000 │ 0.000 │
│  4.000 │ 2.000 │ 1.000 │ 0.000 │ 0.000 │
│  5.000 │ 2.000 │ 1.000 │ 0.000 │ 0.000 │
│  6.000 │ 2.000 │ 0.000 │ 2.000 │ 0.000 │
│  7.000 │ 2.000 │ 0.000 │ 1.000 │ 1.000 │
│  8.000 │ 2.000 │ 1.000 │ 0.000 │ 0.000 │
│  9.000 │ 2.000 │ 1.000 │ 0.000 │ 0.000 │
│ 10.000 │ 2.000 │ 1.000 │ 0.000 │ 0.000 │
└────────┴───────┴───────┴───────┴───────┘


In [67]:
XL = zeros(2)    
YL = zeros(2)      
ZL = zeros(2)  
LL = zeros(2)
CoutL = zeros(2)    
ScL = zeros(2, serM, tserM)
SinL = zeros(serM, tserM)

2×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [69]:
phiL = 2
 
for t in 1:horiz
    println("------------------------------------------------")
    XL[1] = X[t]
    YL[1] = Y[t]
    ZL[1] = Z[t]
    LL[1] = L[t]
    CoutL[1] = Cout[t]
    ScL[1, :, :] = Sc[t, :, :]  # server conveyor status
    
    nL = YM - YL[1] + phiL  # number of empty slots in the queue

    M1 = max(d[t], YM)+10; # must be larger than d[t] and n[t] 
    if d[t] < nL
        b1 = 1
    else
        b1 = 0
    end

    SL = serM
    SstL = sum(ScL[1,:,:], dims=2)
    SlL = SL - sum(SstL)
    SauxL = ones(Bool, serM) - SstL

    M2 = max(X[t], serM)+10; # must be larger than x[k] and Sl[k]
    if X[t]-a[t] < SlL
        b2 = 0
    else
        b2 = 1
    end

    for i in 1:serM
        for j in 1:tserM
            SinL[i, j] = default_input[i, j] .* SauxL[i]
        end
    end
    
    QL = min(d[t], nL)  # demand cannot be larger than number of empty slots in the queue
    CinL = min(X[t]-a[t], SlL)  # Cin cannot be larger than X-a and Sl

    drL = min(d[t]-QL)  # dropped calls cannot be larger than d[n]-nL and d[n]-QL

    ScL[2, :, :] = ScL[1, :, :]*transition_matrix + SinL

    XL[2] = XL[1] + phiL - a[t] - CinL
    YL[2] = YL[1] + QL - phiL
    ZL[2] = ZL[1] + CoutL[1]
    LL[2] = LL[1] + drL + a[t]

    CoutL[2] = sum(ScL[1,:,tserM])  # Cout is the sum of the last conveyor state

    println("Cin: ", CinL)
    println("Sl: ", SlL)
    println("X: ", XL[1])
    println("Y: ", YL[1])
    println("M1: ", M1)
    println("M2: ", M2)
    print("Sc")
    display(ScL[2,:,:])
    print("Sst")
    display(SstL)
    print("Saux")
    display(SauxL)
    print("Sin")
    display(SinL)

    X[t+1] =XL[2];
    Y[t+1] =YL[2];
    Z[t+1] =ZL[2];
    L[t+1] =LL[2];

    n[t] = nL
    Q[t] = QL  
    dr[t] = drL
        
    phi[t] = (phiL);
    Cin[t] = CinL
    Cout[t+1] = CoutL[2]

    S[t] = SL;
    Sl[t] = SlL;
    Sst[t, :] = SstL;
    Sc[t+1, :, :] = ScL[2, :, :]
    Sin[t, :, :] = SinL
    Saux[t, :] = SauxL

    b1_opt[t] = b1
    b2_opt[t] = b2
end


------------------------------------------------
Cin: 2.0
Sl: 2.0
X: 3.0
Y: 2.0
M1: 20.0
M2: 13
Sc

2×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0

Sst

2×1 Matrix{Float64}:
 0.0
 0.0

Saux

...

2×1 Matrix{Float64}:
 1.0
 1.0

Saux

2×1 Matrix{Float64}:
 0.0
 0.0

Sin

2×4 Matrix{Float64}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [62]:
display(Sc[2,:,:])

2×4 Matrix{Bool}:
 0  0  0  0
 0  0  0  0